In [48]:
# Dynamic Pricing for Urban Parking Lots
# Capstone Project - Summer Analytics 2025
!pip install pathway bokeh --quiet  #Installing pathway bokeh

In [49]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [50]:
#Importing bokeh for visualization
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource
import time
from bokeh.plotting import output_file

In [51]:
BASE_PRICE = 10.0  #Base price is 10$
df= pd.read_csv('dataset.csv')
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)  #Converting the Date and Time in appropriate format by combining both.

In [52]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [53]:
# A class to represent each parking record row
class ParkingRecord:
    def __init__(self, timestamp, lot_id, latitude, longitude, capacity, occupancy, queue_length, traffic_level, special_day, vehicle_type):
        self.timestamp = timestamp
        self.lot_id = lot_id
        self.latitude = latitude
        self.longitude = longitude
        self.capacity = capacity
        self.occupancy = occupancy
        self.queue_length = queue_length
        self.traffic_level = traffic_level
        self.special_day = special_day
        self.vehicle_type = vehicle_type

In [54]:
#Assuming the weights of car, bike, truck as per their size
vehicle_type_weights = {
    'car': 1.0,
    'bike': 0.7,
    'truck': 1.3
}


In [55]:
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.io import push_notebook

In [56]:
# Model 1: Linear
def model_1_price(prev_price, occupancy, capacity):
    alpha = 1.0 #Considering value of alpha as positive small number
    return prev_price + alpha * (occupancy / capacity)

# Model 2: Demand-based
# Pricing based on internal demand factors such as occupancy, queue, traffic, etc.
def model_2_price(record):
    alpha, beta, gamma, delta, epsilon, lam = 1.0, 0.5, 0.3, 1.0, 1.2, 0.5  #Appropriately considering values of all parameters which will increase or decrease the demand

    # Map traffic level strings to numerical values
    traffic_mapping = {'low': 1.0, 'average': 2.0, 'high': 3.0}
    traffic_level_numeric = traffic_mapping.get(str(record.traffic_level).lower(), 1.0) # Default to low if not found

    vehicle_weight = vehicle_type_weights.get(str(record.vehicle_type).lower(), 1.0)
    demand = (
            alpha * (float(record.occupancy)) / float(record.capacity)
            + beta * float(record.queue_length)
            - gamma * traffic_level_numeric  # Use the numeric value here
            + delta * float(record.special_day)
            + epsilon * vehicle_weight
        )
    norm_demand = min(max(demand / 10, 0), 1) #To normalize it between 0 and 1
    price = BASE_PRICE * (1 + lam * norm_demand)
    return min(max(price, 0.5 * BASE_PRICE), 2.0 * BASE_PRICE)  #his line is used to restrict the final price within a specific range

# Model 3: Competitive
# Adjusts price based on nearby competitor lots within small geo-radius
# Takes the base price from model_2 and modifies it based on nearby prices

def model_3_price(record, lot_data, all_prices):  #record is an object of the class ParkingRecord, which holds all information about the current data row
    this_lot = lot_data[record.lot_id] # info about the current lot from lot_data, such as its lat(latitude)/lon(longitude)

    competitors = [lot for lot in lot_data.values() if lot['lot_id'] != record.lot_id] # This gives us a list of all other lots (excluding the current one), So we are identifying potential competitors.
    nearby_prices = [all_prices.get(comp['lot_id'], BASE_PRICE)
                     for comp in competitors
                     if abs(comp['lat'] - this_lot['lat']) < 0.01 and abs(comp['lon'] - this_lot['lon']) < 0.01]  # Filters lots that are geographically close (within ~0.01 degrees lat/lon). Gets their current price from all_priceIf no price available, fallback to BASE_PRICE
    base_price = model_2_price(record)  # This calculates the demand-based price (without competition).
    if record.occupancy >= record.capacity and nearby_prices:  # The lot is full (very high demand) AND there are nearby competitors
        return min(base_price, min(nearby_prices))
    elif nearby_prices:  # Your lot is not full but there are nearby competitors
        return max(base_price, sum(nearby_prices) / len(nearby_prices))
    return base_price

In [57]:
!panel serve your_script.py --autoreload

Traceback (most recent call last):
  File "/usr/local/bin/panel", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/panel/command/__init__.py", line 95, in main
    raise e
  File "/usr/local/lib/python3.11/dist-packages/panel/command/__init__.py", line 92, in main
    ret = parsed_args.invoke(parsed_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/panel/command/serve.py", line 772, in invoke
    super().invoke(args)
  File "/usr/local/lib/python3.11/dist-packages/bokeh/command/subcommands/serve.py", line 832, in invoke
    applications = build_single_handler_applications(files, argvs)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bokeh/command/util.py", line 174, in build_single_handler_applications
    application = build_single_handler_application(path, argvs.get(path, []))
                  ^^^^^^^^^^^^^^^^^^

In [58]:
# BOKEH PLOT SETUP
# Initialize Bokeh data source and figure for dynamic pricing visualization
source = ColumnDataSource(data=dict(time=[], price=[]))
p = figure(title="Real-Time Parking Price Stream", x_axis_label='Time', y_axis_label='Price', x_axis_type='datetime')
p.line(x='time', y='price', source=source, line_width=2)

# --- READ AND PROCESS DATA ---
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)
lot_data = {}        # stores metadata of each lot
all_prices = {}      # stores last computed price for each lot
timestamps = []      # times for plotting
prices = []          # prices for plotting

# SIMULATED LIVE STREAM IN BATCHES
# Process first 500 rows in batches and update graph every 50 steps to simulate animation
for index, row in df.iterrows():
    if index >= 500:
        break

# Create record instance from current row
    record = ParkingRecord(
        timestamp=row['timestamp'],
        lot_id=row['SystemCodeNumber'],
        latitude=row['Latitude'],
        longitude=row['Longitude'],
        capacity=row['Capacity'],
        occupancy=row['Occupancy'],
        queue_length=row['QueueLength'],
        traffic_level=row['TrafficConditionNearby'],
        special_day=row['IsSpecialDay'],
        vehicle_type=row['VehicleType']
    )
# Update lot information dictionary

    lot_data[record.lot_id] = {
        'lot_id': record.lot_id,
        'lat': record.latitude,
        'lon': record.longitude
    }
# Compute price using competitive model
    price = model_3_price(record, lot_data, all_prices)
    all_prices[record.lot_id] = price

# Append values for plot
    timestamps.append(record.timestamp)
    prices.append(price)

# Update the plot every 50 steps to simulate movement
    if index % 50 == 0:
        source.data = dict(time=timestamps, price=prices)
        show(p)

# FINAL PLOT
# Show complete price-time curve after simulation

source.data = dict(time=timestamps, price=prices)
show(p)

In [59]:
# --- SETUP LIVE STREAMING PLOT ---
source = ColumnDataSource(data=dict(time=[], price=[]))
p = figure(title="Real-Time Parking Price Stream", x_axis_label='Time', y_axis_label='Price', x_axis_type='datetime')
p.line(x='time', y='price', source=source, line_width=2)
handle = show(p, notebook_handle=True)

# --- READ AND PROCESS DATA ---
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)
lot_data = {}
all_prices = {}
timestamps = []
prices = []

# Real-time streaming loop
for index, row in df.iterrows():
    if index >= 500:
        break

    record = ParkingRecord(
        timestamp=row['timestamp'],
        lot_id=row['SystemCodeNumber'],
        latitude=row['Latitude'],
        longitude=row['Longitude'],
        capacity=row['Capacity'],
        occupancy=row['Occupancy'],
        queue_length=row['QueueLength'],
        traffic_level=row['TrafficConditionNearby'],
        special_day=row['IsSpecialDay'],
        vehicle_type=row['VehicleType']
    )

    lot_data[record.lot_id] = {
        'lot_id': record.lot_id,
        'lat': record.latitude,
        'lon': record.longitude
    }

    price = model_3_price(record, lot_data, all_prices)
    all_prices[record.lot_id] = price

    # Update stream source in chunks
    timestamps.append(record.timestamp)
    prices.append(price)

    if index % 25 == 0:
        delta_window.data = {"t": timestamps.copy(), "price": prices.copy()}
        time.sleep(0.05)

# --- PRICE PLOTTER FOR PANEL ---
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price (Live Simulated)",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

# --- PANEL DISPLAY ---
pn.Column(price_plotter(delta_window)).servable()

Column
    [0] Bokeh(figure)